In [24]:
import os
import openai 

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

import warnings
warnings.filterwarnings("ignore")

In [25]:
import datetime
current_time = datetime.datetime.now().date()
target_time = datetime.date(2024, 6, 12)
if current_time > target_time:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [26]:
import pandas as pd
df = pd.read_csv("Data.csv")

In [27]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


### LLMChain

In [28]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [29]:
llm = ChatOpenAI(model_name=llm_model, temperature=0.9)

In [30]:
prompt = ChatPromptTemplate.from_template(
    "What is name of the biggest competitor that makes {product}?, give name only, no other text."
)

In [31]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

In [32]:
product = df["Product"].iloc[1]
product

'Waterproof Phone Pouch'

In [33]:
chain.run(product=product)



> Entering new LLMChain chain...
Prompt after formatting:
Human: What is name of the biggest competitor that makes Waterproof Phone Pouch?, give name only, no other text.

> Finished chain.


'JOTO'

### SimpleSequentialChain

In [34]:
from langchain.chains import SimpleSequentialChain

In [35]:
llm = ChatOpenAI(model_name=llm_model, temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "What is name of the biggest competitor that makes {product}?, give name only, no other text."
)

chain1 = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True
)     

In [36]:
sencond_prompt = ChatPromptTemplate.from_template(
    """Write a 20 words discription for the following competitors: {competitors}"""
)

chain2 = LLMChain(
    llm=llm,
    prompt=sencond_prompt,
    verbose=True
)


In [37]:
overall_chain = SimpleSequentialChain(
    chains=[chain1, chain2],
    # verbose=True
)   


In [38]:
overall_chain.run(product)



> Entering new LLMChain chain...
Prompt after formatting:
Human: What is name of the biggest competitor that makes Waterproof Phone Pouch?, give name only, no other text.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a 20 words discription for the following competitors: Mpow

> Finished chain.


'Mpow offers affordable and high-quality audio accessories, including headphones, earbuds, and speakers, with a focus on ergonomic design.'

### SequentialChain

In [39]:
from langchain.chains import SequentialChain

In [42]:
llm = ChatOpenAI(model_name=llm_model, temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to Vietnamese: {review}"
)

chain1 = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=False,
    output_key="vietnamese_review"
)   
second_prompt = ChatPromptTemplate.from_template(
    """What language is the following review: {vietnamese_review}"""
)

chain2 = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=False,
    output_key="language"
)

third_prompt = ChatPromptTemplate.from_template(
    """Summarize the following review in one sentence: {vietnamese_review} in {language} language"""
)

chain3 = LLMChain(
    llm=llm,
    prompt=third_prompt,
    verbose=False,
    output_key="vietnamese_summary"
)

fourth_prompt = ChatPromptTemplate.from_template(
        "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {vietnamese_summary}\n\nLanguage: {language}"
)

chain4 = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=False,
    output_key="follow_up_response"
)

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["review"],
    output_variables=["vietnamese_review", "language", "vietnamese_summary", "follow_up_response"],
    verbose=False
)

review = df["Review"].iloc[1]
overall_chain(review)

{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'vietnamese_review': 'Tôi rất thích túi chống nước, mặc dù miệng túi làm từ nhựa cứng. Tôi không biết liệu có dễ vỡ hay không. Nhưng khi tôi đặt điện thoại vào túi, tôi không thể bật máy lên được.',
 'language': 'This review is in Vietnamese.',
 'vietnamese_summary': 'I really like the waterproof bag, even though the mouth of the bag is made of hard plastic and I am unsure if it is easy to break, but when I put my phone in the bag, I cannot turn it on.',
 'follow_up_response': 'Cảm ơn bạn đã chia sẻ nhận xét về túi chống nước. Đúng là túi rất tiện lợi, nhưng phần miệng làm bằng nhựa cứng đúng là khiến mình lo lắng về tính bền vững của nó. Bạn nên thử kiểm tra xem có thể do cách đặt điện thoại vào túi không chính xác khiến không thể bật máy lên được. Hy vọng bạn sẽ tìm ra cách sử dụng hiệu quả hơn 